In [7]:
# https://python.langchain.com/docs/integrations/document_loaders/pypdfloader/
# https://python.langchain.com/docs/integrations/document_loaders/#pdfs
# https://python.langchain.com/api_reference/text_splitters/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html#recursivecharactertextsplitter
# https://python.langchain.com/docs/integrations/text_embedding/
# https://python.langchain.com/docs/how_to/document_loader_pdf/

from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [8]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

In [9]:
loader = PyPDFLoader("./tictactoe_user_stories.pdf")

In [10]:
pages = loader.load_and_split()

In [11]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [12]:

vector_store = InMemoryVectorStore.from_documents(pages, embeddings)
retriever = vector_store.as_retriever()

In [13]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [14]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [15]:
template = """You are a helpful AI assistant. 
Be factual in your response.
Respond the folowing question: {question} only from below context: {context}.
If you don't know the answer, just say that you don't know, don't try to make up an answer."""

In [16]:
prompt = PromptTemplate.from_template(template)

In [20]:
chain = (
  {"context": retriever | format_docs, "question": RunnablePassthrough()}
  | prompt  | llm | StrOutputParser()
)

In [21]:
chain.invoke( "What is the first user story?")

'The first user story from the context provided is: "As a Google user, I want to log in using my Google account, so that my identity is authenticated securely and my progress can be tracked."'

In [23]:
chain.invoke( "Which credentials I have to use to login?")

'As a user, you should use your Google account credentials to log in to the application.'

In [24]:
chain.invoke( "I am blind, can I play the game?")

'Based on the user stories provided, the game should be accessible to users with disabilities, including those who are blind. The application should meet WCAG 2.2 Level A compliance, ensuring that the interface is accessible to users with disabilities. This means that as a blind user, you should be able to play the game with the necessary accessibility features in place.'